## Import

In [3]:
import random
import os

import numpy as np
import pandas as pd

# 변동이 적은 특징을 제거하는데 사용하는 모듈
from sklearn.feature_selection import VarianceThreshold

# 데이터 셋을 학습, 검증, 테스트 셋으로 분할하는 데 사용하는 모듈
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

# Pytorch에서 데이터셋 처리 및 배치 생성에 사용하는 모듈
from torch.utils.data import Dataset, DataLoader

# rdkit : 화학 구조 및 데이터 처리를 위한 라이브러리
# DataStructs : 데이터 구조 처리
# PandasTools, AllChem : 화학 물질 처리
from rdkit import DataStructs
from rdkit.Chem import PandasTools, AllChem

In [4]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)

seed_everything(42) # Seed 고정

## Pre-Processing

In [8]:
train = pd.read_csv('open (1)/train.csv')
test = pd.read_csv('open (1)/test.csv')

In [9]:
# SMILES 문자열을 사용하여 Molecule 열을 데이터프레임에 train, test 각각 추가
PandasTools.AddMoleculeColumnToFrame(train,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(test,'SMILES','Molecule')

In [10]:
# 주어진 mol을 Morgan fingerprint로 변환
# Morgan fingerprint : 화학 물질의 구조와 성질을 나타내는 벡터 형식의 표현
# GetHashedMorganFingerprint() : 화학물질 mol을 입력받아 해당 물질의 fingerprint로 반환

def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

In [11]:
# FPs column 추가 -> Morgan Fingerprint를 계산한 값

train["FPs"] = train.Molecule.apply(mol2fp)
test["FPs"] = test.Molecule.apply(mol2fp)

In [12]:
# 사용할 column만 추출

train = train[['FPs','MLM', 'HLM']]
test = test[['FPs']]

## Custom Dataset

In [13]:
class CustomDataset(Dataset):
    def __init__(self, df, target, transform, is_test=False):
        self.df = df
        self.target = target # HLM or MLM
        self.is_test = is_test # train,valid / test

        self.feature_select = transform   # transform = VarianceThreshold
        if not self.is_test: 
            self.fp = self.feature_select.fit_transform(np.stack(df['FPs']))
        else: # valid or test
            self.fp = self.feature_select.transform(np.stack(df['FPs']))

    def __getitem__(self, index):
        fp = self.fp[index]
        if not self.is_test: # test가 아닌 경우(label 존재)
            label = self.df[self.target][index]
            return torch.tensor(fp).float(), torch.tensor(label).float().unsqueeze(dim=-1) # feature, label

        else: # test인 경우
            return torch.tensor(fp).float() # feature
        
    def __len__(self):
        return len(self.df)

In [14]:
transform = VarianceThreshold(threshold=0.05)    # 변동이 0.05 이하인 특성을 제거

train_MLM = CustomDataset(df=train, target='MLM', transform=transform, is_test=False)
train_HLM = CustomDataset(df=train, target='HLM', transform=transform, is_test=False)

# 모델에 입력될 특성의 수
input_size = train_MLM.fp.shape[1]
input_size

251

In [15]:
# Hyperparameter
CFG = {'BATCH_SIZE': 256,
       'EPOCHS': 1000,
       'INPUT_SIZE': input_size,
       'HIDDEN_SIZE': 1024,
       'OUTPUT_SIZE': 1,
       'DROPOUT_RATE': 0.8,
       'LEARNING_RATE': 0.001}

In [16]:
# train,valid split
train_MLM_dataset, valid_MLM_dataset = train_test_split(train_MLM, test_size=0.2, random_state=42)
train_HLM_dataset, valid_HLM_dataset = train_test_split(train_HLM, test_size=0.2, random_state=42)

In [17]:
train_MLM_loader = DataLoader(dataset=train_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_MLM_loader = DataLoader(dataset=valid_MLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)


train_HLM_loader = DataLoader(dataset=train_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=True)

valid_HLM_loader = DataLoader(dataset=valid_HLM_dataset,
                              batch_size=CFG['BATCH_SIZE'],
                              shuffle=False)

## Model

In [18]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_rate, out_size):
        super(Net, self).__init__()
        
        # fc 레이어 3개와 출력 레이어
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc_out = nn.Linear(hidden_size, out_size)
        
        # 정규화
        self.ln1 = nn.LayerNorm(hidden_size)
        self.ln2 = nn.LayerNorm(hidden_size)
        self.ln3 = nn.LayerNorm(hidden_size)        
        
        # 활성화 함수
        self.activation = nn.LeakyReLU()
        
        # Dropout
        self.dropout = nn.Dropout(dropout_rate)
     
    def forward(self, x):
        out = self.fc1(x)
        out = self.ln1(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        out = self.fc2(out)
        out = self.ln2(out)
        out = self.activation(out)
        out = self.dropout(out)
        
        out = self.fc3(out)
        out = self.ln3(out)
        out = self.activation(out)
        out = self.dropout(out)

        out = self.fc_out(out)
        return out

In [19]:
model_MLM = Net(CFG['INPUT_SIZE'],CFG['HIDDEN_SIZE'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE'])
model_HLM = Net(CFG['INPUT_SIZE'],CFG['HIDDEN_SIZE'],CFG['DROPOUT_RATE'],CFG['OUTPUT_SIZE'])

In [20]:
criterion = nn.MSELoss()
optimizer_MLM = torch.optim.Adam(model_MLM.parameters(), lr=CFG['LEARNING_RATE'])
optimizer_HLM = torch.optim.Adam(model_HLM.parameters(), lr=CFG['LEARNING_RATE'])

## Training

In [21]:
def train(train_loader, valid_loader, model, criterion, optimizer, epochs):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
            output = model(inputs)
            loss = criterion(output, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        if epoch % 100 == 0:
            valid_loss = 0
            with torch.no_grad():
                for inputs, targets in valid_loader:
                    output = model(inputs)
                    loss = criterion(output, targets)
                    valid_loss += loss.item()
                    
            print(f'Epoch: {epoch}/{epochs}, Train Loss: {running_loss/len(train_loader)}, Valid Loss: {valid_loss/len(valid_HLM_loader)}')
            
            model.train()
    
    return model

In [22]:
print("Training Start: MLM")
model_MLM = train(train_MLM_loader, valid_MLM_loader, model_MLM, criterion, optimizer_MLM, epochs=CFG['EPOCHS'])

print("Training Start: HLM")
model_HLM = train(train_HLM_loader, valid_HLM_loader, model_HLM, criterion, optimizer_HLM, epochs=CFG['EPOCHS'])  

Training Start: MLM
Epoch: 0/1000, Train Loss: 2092.469149502841, Valid Loss: 1900.7298177083333
Epoch: 100/1000, Train Loss: 315.07490955699575, Valid Loss: 1596.1493326822917
Epoch: 200/1000, Train Loss: 179.85753839666194, Valid Loss: 1462.0851643880208
Epoch: 300/1000, Train Loss: 142.6571468006481, Valid Loss: 1484.1102294921875
Epoch: 400/1000, Train Loss: 109.9487075805664, Valid Loss: 1524.2877197265625
Epoch: 500/1000, Train Loss: 104.08066836270419, Valid Loss: 1559.7952473958333
Epoch: 600/1000, Train Loss: 85.91273706609553, Valid Loss: 1561.7167561848958
Epoch: 700/1000, Train Loss: 81.01566210660067, Valid Loss: 1538.984130859375
Epoch: 800/1000, Train Loss: 75.30524097789417, Valid Loss: 1555.6198323567708
Epoch: 900/1000, Train Loss: 66.95597908713601, Valid Loss: 1592.1287434895833
Training Start: HLM
Epoch: 0/1000, Train Loss: 3230.5881569602275, Valid Loss: 2635.5455729166665
Epoch: 100/1000, Train Loss: 400.2231805974787, Valid Loss: 1366.9431559244792
Epoch: 200/10

## Inference

In [23]:
test_MLM = CustomDataset(df=test, target=None, transform=transform, is_test=True)
test_HLM = CustomDataset(df=test, target=None, transform=transform, is_test=True)

test_MLM_loader = DataLoader(dataset=test_MLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

test_HLM_loader = DataLoader(dataset=test_HLM,
                             batch_size=CFG['BATCH_SIZE'],
                             shuffle=False)

In [24]:
def inference(test_loader, model):
    model.eval()
    preds = []
    
    with torch.no_grad():
        for inputs in test_loader:
            output = model(inputs)
            preds.extend(output.cpu().numpy().flatten().tolist())
    
    return preds

In [25]:
predictions_MLM = inference(test_MLM_loader, model_MLM)
predictions_HLM = inference(test_HLM_loader, model_HLM)

## Submission

In [28]:
submission = pd.read_csv('open (1)/sample_submission.csv')
submission

,id,MLM,HLM
0,TEST_000,0,0
1,TEST_001,0,0
2,TEST_002,0,0
3,TEST_003,0,0
4,TEST_004,0,0
...,...,...,...
478,TEST_478,0,0
479,TEST_479,0,0
480,TEST_480,0,0
481,TEST_481,0,0


In [29]:
submission['MLM'] = predictions_MLM
submission['HLM'] = predictions_HLM
submission

,id,MLM,HLM
0,TEST_000,5.561823,7.321818
1,TEST_001,49.405273,84.464272
2,TEST_002,26.698666,51.190384
3,TEST_003,27.090666,56.688736
4,TEST_004,26.869804,92.281189
...,...,...,...
478,TEST_478,57.758179,87.065834
479,TEST_479,76.982445,97.395988
480,TEST_480,29.691576,67.671494
481,TEST_481,38.914825,66.260643


In [30]:
submission.to_csv('baseline_submission.csv', index=False)